# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
 
    source_id_text = []
    for source_sentence in source_text.split('\n'):
#         print("--{}--".format(source_sentence))
        source_id_text.append([source_vocab_to_int[word] for word in source_sentence.split(' ') if len(word) != 0])

    target_id_text = []
    for target_sentence in target_text.split('\n'):
#         print("--{}--".format(target_sentence))
        target_sentence=target_sentence+" <EOS>"
        target_id_text.append([target_vocab_to_int[word] for word in target_sentence.split(' ') if len(word) != 0])
        
    return source_id_text, target_id_text


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input_ = tf.placeholder(tf.int32, [None, None], name = "input")
    targets_ = tf.placeholder(tf.int32, [None, None], name = "targets")
    learning_rate_ = tf.placeholder(tf.float32, name = "learning_rate")
    keep_prob_ = tf.placeholder(tf.float32, name = "keep_prob")
    target_sequence_length_ = tf.placeholder(tf.int32, [None], name = "target_sequence_length")
    max_target_sequence_length_ = tf.reduce_max(target_sequence_length_)    
    source_sequence_length_ = tf.placeholder(tf.int32, [None], name = "source_sequence_length")    
  
    return input_, targets_, learning_rate_, keep_prob_, target_sequence_length_, max_target_sequence_length_, source_sequence_length_


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    
    # batch_size: How Many Batches
    # Sequence Length: How Many Characters in a Sentence
    
    # Find the ID of Go
    go_id = target_vocab_to_int['<GO>']
    
    # Remove the Last Char, Not Include [-1]
    target_data_no_last = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    
    # Find the ID of Go, Generate a tensor likes: [[go_id], [go_id], [go_id]]
    go = tf.fill([batch_size, 1], go_id)

    # Concat: [[go_id, 1, 2], [go_id, 3, 4], [go_id, 5, 6]]    
    preprocessed_target_data = tf.concat([go, target_data_no_last], 1)
    
    return preprocessed_target_data

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    
    # Embed the encoder input
    source_embedding = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell(
                            [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(
                                    rnn_size,
                                    initializer = tf.random_uniform_initializer(-0.1, 0.1, seed=1234)
                                    ), 
                                output_keep_prob=keep_prob) for _ in range(num_layers)])
    rnn_output, rnn_state = tf.nn.dynamic_rnn(stacked_lstm_cell, 
                                             source_embedding, 
                                             sequence_length = source_sequence_length, 
                                             dtype =tf.float32)

    return rnn_output, rnn_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function    
    
    # Input will be the decoder's embedded input
    # Length is the target sequence length
    helper_ = tf.contrib.seq2seq.TrainingHelper(
                    inputs = dec_embed_input,
                    sequence_length = target_sequence_length)
    
    # Cells are the decoder's LSTM cells
    # initial_state is the encoder's cell states
    # Output layer is optional, but here we have output layer
    decoder_ = tf.contrib.seq2seq.BasicDecoder(
                    cell = dec_cell,
                    helper = helper_,
                    initial_state = encoder_state,
                    output_layer = output_layer
                    )
    
    # maximum_iterations is set to be the maximum length of the sequence    
    outputs, _ = tf.contrib.seq2seq.dynamic_decode(
                   decoder = decoder_,
                   output_time_major = False,
                   impute_finished = True,
                   maximum_iterations = max_summary_length)
            
    return outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    
    # Input will be the decoder's embedded input
    # Length is the target sequence length
    helper_ = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                    embedding = dec_embeddings,
                    start_tokens=tf.tile([start_of_sequence_id], [batch_size]),
                    end_token = end_of_sequence_id)
    
    # Cells are the decoder's LSTM cells
    # initial_state is the encoder's cell states
    # Output layer is optional, but here we have output layer
    decoder_ = tf.contrib.seq2seq.BasicDecoder(
                    cell = dec_cell,
                    helper = helper_,
                    initial_state = encoder_state,
                    output_layer = output_layer
                    )
    
    # maximum_iterations is set to be the maximum length of the sequence    
    outputs, _ = tf.contrib.seq2seq.dynamic_decode(
                   decoder = decoder_,
                   output_time_major = False,
                   impute_finished = True,
                   maximum_iterations = max_target_sequence_length)
            
    return outputs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    # Embed the target sequences
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embeddings_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    


    # Construct the decoder LSTM cell    
    stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell(
                            [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(
                                    rnn_size,
                                    initializer = tf.random_uniform_initializer(-0.1, 0.1, seed=1234)
                                    ), 
                                output_keep_prob=keep_prob) for _ in range(num_layers)])

    
    # Create an output layer     
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))   
    

    # Use the your decoding_layer_train
    # For Training, Decoder Input is from Given Sequence
    
    with tf.variable_scope("decode"):
        training_decoder_output = decoding_layer_train(encoder_state, stacked_lstm_cell, dec_embeddings_input, 
                                                       target_sequence_length, max_target_sequence_length, 
                                                       output_layer, keep_prob)
 

    # Use your decoding_layer_infer
    # For Prediction, Decoder Input is from Last Decoder Prediction Output
    with tf.variable_scope("decode", reuse=True):
        inference_decoder_output = decoding_layer_infer(encoder_state, stacked_lstm_cell, dec_embeddings, 
                                                        target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], 
                                                        max_target_sequence_length,
                                                        target_vocab_size, output_layer, batch_size, keep_prob)     
    
    
    
    return training_decoder_output, inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    _, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob,  
                   source_sequence_length, source_vocab_size, enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    training_decoder_output, inference_decoder_output = \
        decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, 
                       rnn_size, num_layers, target_vocab_to_int, target_vocab_size, 
                       batch_size, keep_prob, dec_embedding_size)
    
    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [18]:
# Number of Epochs
epochs = 30
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 200
decoding_embedding_size = 200
# Learning Rate
learning_rate = 0.01
# Dropout Keep Probability
keep_probability = 0.8
display_step = 10

### Build the Graph
Build the graph using the neural network you implemented.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   10/538 - Train Accuracy: 0.3045, Validation Accuracy: 0.3793, Loss: 3.3336
Epoch   0 Batch   20/538 - Train Accuracy: 0.3664, Validation Accuracy: 0.4048, Loss: 2.8061
Epoch   0 Batch   30/538 - Train Accuracy: 0.4215, Validation Accuracy: 0.4698, Loss: 2.5725
Epoch   0 Batch   40/538 - Train Accuracy: 0.4973, Validation Accuracy: 0.5020, Loss: 2.0273
Epoch   0 Batch   50/538 - Train Accuracy: 0.4945, Validation Accuracy: 0.5270, Loss: 1.8720
Epoch   0 Batch   60/538 - Train Accuracy: 0.4760, Validation Accuracy: 0.5265, Loss: 1.5308
Epoch   0 Batch   70/538 - Train Accuracy: 0.4448, Validation Accuracy: 0.4856, Loss: 1.1575
Epoch   0 Batch   80/538 - Train Accuracy: 0.3320, Validation Accuracy: 0.3819, Loss: 1.0557
Epoch   0 Batch   90/538 - Train Accuracy: 0.5113, Validation Accuracy: 0.5201, Loss: 0.8978
Epoch   0 Batch  100/538 - Train Accuracy: 0.5395, Validation Accuracy: 0.5637, Loss: 0.8220
Epoch   0 Batch  110/538 - Train Accuracy: 0.5361, Validation Accuracy

Epoch   1 Batch  370/538 - Train Accuracy: 0.9516, Validation Accuracy: 0.9281, Loss: 0.0484
Epoch   1 Batch  380/538 - Train Accuracy: 0.9314, Validation Accuracy: 0.9274, Loss: 0.0479
Epoch   1 Batch  390/538 - Train Accuracy: 0.9375, Validation Accuracy: 0.9384, Loss: 0.0461
Epoch   1 Batch  400/538 - Train Accuracy: 0.9395, Validation Accuracy: 0.9428, Loss: 0.0548
Epoch   1 Batch  410/538 - Train Accuracy: 0.9432, Validation Accuracy: 0.9386, Loss: 0.0512
Epoch   1 Batch  420/538 - Train Accuracy: 0.9420, Validation Accuracy: 0.9293, Loss: 0.0494
Epoch   1 Batch  430/538 - Train Accuracy: 0.9164, Validation Accuracy: 0.9405, Loss: 0.0488
Epoch   1 Batch  440/538 - Train Accuracy: 0.9326, Validation Accuracy: 0.9354, Loss: 0.0534
Epoch   1 Batch  450/538 - Train Accuracy: 0.9293, Validation Accuracy: 0.9421, Loss: 0.0661
Epoch   1 Batch  460/538 - Train Accuracy: 0.9308, Validation Accuracy: 0.9329, Loss: 0.0493
Epoch   1 Batch  470/538 - Train Accuracy: 0.9330, Validation Accuracy

Epoch   3 Batch  200/538 - Train Accuracy: 0.9828, Validation Accuracy: 0.9563, Loss: 0.0226
Epoch   3 Batch  210/538 - Train Accuracy: 0.9580, Validation Accuracy: 0.9501, Loss: 0.0354
Epoch   3 Batch  220/538 - Train Accuracy: 0.9459, Validation Accuracy: 0.9492, Loss: 0.0338
Epoch   3 Batch  230/538 - Train Accuracy: 0.9508, Validation Accuracy: 0.9419, Loss: 0.0404
Epoch   3 Batch  240/538 - Train Accuracy: 0.9596, Validation Accuracy: 0.9535, Loss: 0.0371
Epoch   3 Batch  250/538 - Train Accuracy: 0.9566, Validation Accuracy: 0.9528, Loss: 0.0338
Epoch   3 Batch  260/538 - Train Accuracy: 0.9466, Validation Accuracy: 0.9446, Loss: 0.0381
Epoch   3 Batch  270/538 - Train Accuracy: 0.9488, Validation Accuracy: 0.9421, Loss: 0.0301
Epoch   3 Batch  280/538 - Train Accuracy: 0.9624, Validation Accuracy: 0.9359, Loss: 0.0287
Epoch   3 Batch  290/538 - Train Accuracy: 0.9551, Validation Accuracy: 0.9528, Loss: 0.0346
Epoch   3 Batch  300/538 - Train Accuracy: 0.9606, Validation Accuracy

Epoch   5 Batch   30/538 - Train Accuracy: 0.9537, Validation Accuracy: 0.9474, Loss: 0.0335
Epoch   5 Batch   40/538 - Train Accuracy: 0.9586, Validation Accuracy: 0.9517, Loss: 0.0240
Epoch   5 Batch   50/538 - Train Accuracy: 0.9467, Validation Accuracy: 0.9524, Loss: 0.0310
Epoch   5 Batch   60/538 - Train Accuracy: 0.9617, Validation Accuracy: 0.9668, Loss: 0.0327
Epoch   5 Batch   70/538 - Train Accuracy: 0.9637, Validation Accuracy: 0.9657, Loss: 0.0292
Epoch   5 Batch   80/538 - Train Accuracy: 0.9727, Validation Accuracy: 0.9549, Loss: 0.0272
Epoch   5 Batch   90/538 - Train Accuracy: 0.9641, Validation Accuracy: 0.9569, Loss: 0.0382
Epoch   5 Batch  100/538 - Train Accuracy: 0.9828, Validation Accuracy: 0.9618, Loss: 0.0234
Epoch   5 Batch  110/538 - Train Accuracy: 0.9662, Validation Accuracy: 0.9602, Loss: 0.0303
Epoch   5 Batch  120/538 - Train Accuracy: 0.9691, Validation Accuracy: 0.9634, Loss: 0.0237
Epoch   5 Batch  130/538 - Train Accuracy: 0.9611, Validation Accuracy

Epoch   6 Batch  390/538 - Train Accuracy: 0.9656, Validation Accuracy: 0.9672, Loss: 0.0241
Epoch   6 Batch  400/538 - Train Accuracy: 0.9628, Validation Accuracy: 0.9634, Loss: 0.0326
Epoch   6 Batch  410/538 - Train Accuracy: 0.9719, Validation Accuracy: 0.9641, Loss: 0.0303
Epoch   6 Batch  420/538 - Train Accuracy: 0.9537, Validation Accuracy: 0.9600, Loss: 0.0285
Epoch   6 Batch  430/538 - Train Accuracy: 0.9527, Validation Accuracy: 0.9529, Loss: 0.0302
Epoch   6 Batch  440/538 - Train Accuracy: 0.9662, Validation Accuracy: 0.9476, Loss: 0.0328
Epoch   6 Batch  450/538 - Train Accuracy: 0.9356, Validation Accuracy: 0.9604, Loss: 0.0393
Epoch   6 Batch  460/538 - Train Accuracy: 0.9622, Validation Accuracy: 0.9675, Loss: 0.0294
Epoch   6 Batch  470/538 - Train Accuracy: 0.9665, Validation Accuracy: 0.9517, Loss: 0.0294
Epoch   6 Batch  480/538 - Train Accuracy: 0.9667, Validation Accuracy: 0.9638, Loss: 0.0292
Epoch   6 Batch  490/538 - Train Accuracy: 0.9606, Validation Accuracy

...

Epoch  21 Batch  450/538 - Train Accuracy: 0.9455, Validation Accuracy: 0.9531, Loss: 0.0485
Epoch  21 Batch  460/538 - Train Accuracy: 0.9470, Validation Accuracy: 0.9645, Loss: 0.0413
Epoch  21 Batch  470/538 - Train Accuracy: 0.9585, Validation Accuracy: 0.9506, Loss: 0.0343
Epoch  21 Batch  480/538 - Train Accuracy: 0.9613, Validation Accuracy: 0.9487, Loss: 0.0347
Epoch  21 Batch  490/538 - Train Accuracy: 0.9596, Validation Accuracy: 0.9535, Loss: 0.0323
Epoch  21 Batch  500/538 - Train Accuracy: 0.9661, Validation Accuracy: 0.9393, Loss: 0.0322
Epoch  21 Batch  510/538 - Train Accuracy: 0.9637, Validation Accuracy: 0.9357, Loss: 0.0360
Epoch  21 Batch  520/538 - Train Accuracy: 0.9520, Validation Accuracy: 0.9498, Loss: 0.0419
Epoch  21 Batch  530/538 - Train Accuracy: 0.9525, Validation Accuracy: 0.9583, Loss: 0.0382
Epoch  22 Batch   10/538 - Train Accuracy: 0.9465, Validation Accuracy: 0.9508, Loss: 0.0421
Epoch  22 Batch   20/538 - Train Accuracy: 0.9591, Validation Accuracy

Epoch  23 Batch  280/538 - Train Accuracy: 0.9446, Validation Accuracy: 0.9437, Loss: 0.0358
Epoch  23 Batch  290/538 - Train Accuracy: 0.9639, Validation Accuracy: 0.9599, Loss: 0.0363
Epoch  23 Batch  300/538 - Train Accuracy: 0.9660, Validation Accuracy: 0.9467, Loss: 0.0420
Epoch  23 Batch  310/538 - Train Accuracy: 0.9594, Validation Accuracy: 0.9542, Loss: 0.0435
Epoch  23 Batch  320/538 - Train Accuracy: 0.9487, Validation Accuracy: 0.9597, Loss: 0.0363
Epoch  23 Batch  330/538 - Train Accuracy: 0.9626, Validation Accuracy: 0.9535, Loss: 0.0312
Epoch  23 Batch  340/538 - Train Accuracy: 0.9533, Validation Accuracy: 0.9624, Loss: 0.0396
Epoch  23 Batch  350/538 - Train Accuracy: 0.9580, Validation Accuracy: 0.9538, Loss: 0.0416
Epoch  23 Batch  360/538 - Train Accuracy: 0.9533, Validation Accuracy: 0.9625, Loss: 0.0392
Epoch  23 Batch  370/538 - Train Accuracy: 0.9604, Validation Accuracy: 0.9558, Loss: 0.0379
Epoch  23 Batch  380/538 - Train Accuracy: 0.9514, Validation Accuracy

Epoch  25 Batch  110/538 - Train Accuracy: 0.9516, Validation Accuracy: 0.9423, Loss: 0.0372
Epoch  25 Batch  120/538 - Train Accuracy: 0.9424, Validation Accuracy: 0.9476, Loss: 0.0351
Epoch  25 Batch  130/538 - Train Accuracy: 0.9537, Validation Accuracy: 0.9423, Loss: 0.0444
Epoch  25 Batch  140/538 - Train Accuracy: 0.9311, Validation Accuracy: 0.9458, Loss: 0.0521
Epoch  25 Batch  150/538 - Train Accuracy: 0.9654, Validation Accuracy: 0.9471, Loss: 0.0410
Epoch  25 Batch  160/538 - Train Accuracy: 0.9500, Validation Accuracy: 0.9450, Loss: 0.0385
Epoch  25 Batch  170/538 - Train Accuracy: 0.9462, Validation Accuracy: 0.9522, Loss: 0.0447
Epoch  25 Batch  180/538 - Train Accuracy: 0.9507, Validation Accuracy: 0.9473, Loss: 0.0482
Epoch  25 Batch  190/538 - Train Accuracy: 0.9462, Validation Accuracy: 0.9393, Loss: 0.0601
Epoch  25 Batch  200/538 - Train Accuracy: 0.9678, Validation Accuracy: 0.9503, Loss: 0.0370
Epoch  25 Batch  210/538 - Train Accuracy: 0.9492, Validation Accuracy

Epoch  26 Batch  470/538 - Train Accuracy: 0.9561, Validation Accuracy: 0.9522, Loss: 0.0417
Epoch  26 Batch  480/538 - Train Accuracy: 0.9650, Validation Accuracy: 0.9553, Loss: 0.0451
Epoch  26 Batch  490/538 - Train Accuracy: 0.9585, Validation Accuracy: 0.9560, Loss: 0.0379
Epoch  26 Batch  500/538 - Train Accuracy: 0.9537, Validation Accuracy: 0.9542, Loss: 0.0326
Epoch  26 Batch  510/538 - Train Accuracy: 0.9589, Validation Accuracy: 0.9595, Loss: 0.0334
Epoch  26 Batch  520/538 - Train Accuracy: 0.9418, Validation Accuracy: 0.9648, Loss: 0.0428
Epoch  26 Batch  530/538 - Train Accuracy: 0.9439, Validation Accuracy: 0.9531, Loss: 0.0423
Epoch  27 Batch   10/538 - Train Accuracy: 0.9547, Validation Accuracy: 0.9471, Loss: 0.0412
Epoch  27 Batch   20/538 - Train Accuracy: 0.9529, Validation Accuracy: 0.9471, Loss: 0.0411
Epoch  27 Batch   30/538 - Train Accuracy: 0.9352, Validation Accuracy: 0.9414, Loss: 0.0492
Epoch  27 Batch   40/538 - Train Accuracy: 0.9542, Validation Accuracy

Epoch  28 Batch  300/538 - Train Accuracy: 0.9550, Validation Accuracy: 0.9517, Loss: 0.0405
Epoch  28 Batch  310/538 - Train Accuracy: 0.9773, Validation Accuracy: 0.9446, Loss: 0.0413
Epoch  28 Batch  320/538 - Train Accuracy: 0.9557, Validation Accuracy: 0.9473, Loss: 0.0418
Epoch  28 Batch  330/538 - Train Accuracy: 0.9537, Validation Accuracy: 0.9432, Loss: 0.0409
Epoch  28 Batch  340/538 - Train Accuracy: 0.9600, Validation Accuracy: 0.9638, Loss: 0.0406
Epoch  28 Batch  350/538 - Train Accuracy: 0.9576, Validation Accuracy: 0.9728, Loss: 0.0479
Epoch  28 Batch  360/538 - Train Accuracy: 0.9506, Validation Accuracy: 0.9680, Loss: 0.0359
Epoch  28 Batch  370/538 - Train Accuracy: 0.9477, Validation Accuracy: 0.9529, Loss: 0.0405
Epoch  28 Batch  380/538 - Train Accuracy: 0.9451, Validation Accuracy: 0.9567, Loss: 0.0384
Epoch  28 Batch  390/538 - Train Accuracy: 0.9503, Validation Accuracy: 0.9569, Loss: 0.0451
Epoch  28 Batch  400/538 - Train Accuracy: 0.9608, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [24]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    print(sentence)
    unk = vocab_to_int['<UNK>']
    
    return [vocab_to_int.get(word, unk) for word in sentence.lower().split(' ')]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

this is a test sentence
Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [25]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


he saw a old yellow truck .
INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [196, 122, 105, 208, 93, 129, 182]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [77, 317, 176, 318, 329, 284, 99, 166, 1]
  French Words: il a vu un vieux camion jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.